In [1]:
import d2l
from mxnet import gluon, nd
from mxnet.gluon import nn

def nin_block(num_channels, kernel_size, strides, padding):
    blk = nn.Sequential()
    blk.add(nn.Conv2D(num_channels, kernel_size, strides, padding, activation='relu'),
            nn.Conv2D(num_channels, kernel_size=1, activation='relu'),
            nn.Conv2D(num_channels, kernel_size=1, activation='relu'))
    return blk

3 concolution işlemi bir block oluşturmaktadır. Convolution filteleri verilerek block yapısı oluşturulmuştur.
Kernel_size değerlerinin 1 olduğuna dikkat ediniz.

In [3]:
net = nn.Sequential()
net.add(nin_block(96, kernel_size=11, strides=4, padding=0),
        nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(256, kernel_size=5, strides=1, padding=2),
        nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(384, kernel_size=3, strides=1, padding=1),
        nn.MaxPool2D(pool_size=3, strides=2),
        nn.Dropout(0.5),
        nin_block(10, kernel_size=3, strides=1, padding=1),
        nn.GlobalAvgPool2D(),
        nn.Flatten())

Block işlemleri yapılmış ve her bloğun ardından mimari gereği max pooling gerçekleştirilmiştir .

In [4]:
X = nd.random.uniform(shape=(1, 1, 224, 224))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential6 output shape:	 (1, 96, 54, 54)
pool4 output shape:	 (1, 96, 26, 26)
sequential7 output shape:	 (1, 256, 26, 26)
pool5 output shape:	 (1, 256, 12, 12)
sequential8 output shape:	 (1, 384, 12, 12)
pool6 output shape:	 (1, 384, 5, 5)
dropout1 output shape:	 (1, 384, 5, 5)
sequential9 output shape:	 (1, 10, 5, 5)
pool7 output shape:	 (1, 10, 1, 1)
flatten1 output shape:	 (1, 10)


Datasetin boyutu 224x224 olrak verilmiştir.

In [ ]:
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch5(net, train_iter, test_iter, num_epochs, lr)